## Loading in the dataset

In [ ]:
import pandas as pd
import numpy as np
import re
df = pd.read_csv("Hourly_Elec_Full_Finished_compleet.xls", sep=',', low_memory=False)

## Column naming consistency 

In [ ]:
# making lowercase column names for consistency
df.columns = [c.lower() for c in df.columns]

# ensuring datetime_hour column exists and is datetime
df['datetime_hour'] = pd.to_datetime(df['datetime_hour'])


## Filling missing values of Spotprices and Regional energy production

In [ ]:
#Resolving problem of having energy spotprice and NaNs of region production to 0
if 'spot_price_elec_mw' in df.columns:
    df['spot_price_elec_mw'] = df.groupby('datetime_hour')['spot_price_elec_mw'].transform(
        lambda x: x.fillna(x.mean()))
    df['spot_price_elec_mw'] = df['spot_price_elec_mw'].ffill().bfill()

# Filling energy columns with NaNs with 0
energy_columns = ['thermique', 'nucléaire', 'eolien', 'solaire', 'hydraulique', 'pompage', 'bioénergies']
energy_columns = [c.lower() for c in energy_columns]
for col in energy_columns:
    if col in df.columns:
        df[col] = df[col].fillna(0)

## Feature Engineering

In [ ]:
#Feature engineering for additional features
df['hour'] = df['datetime_hour'].dt.hour
df['day_of_week'] = df['datetime_hour'].dt.dayofweek
df['month'] = df['datetime_hour'].dt.month
df['is_weekend'] = (df['day_of_week'] >= 5).astype('uint8')
df['week_of_year'] = df['datetime_hour'].dt.isocalendar().week.astype('int8')

#Creating lag  and rolling mean featurss of spot prices
BASE_PRICE = 36.56

# replacing the first missing values of spot price with the base price 
def safe_lag(series, periods):
    lagged = series.shift(periods)
    lagged.iloc[:periods] = BASE_PRICE  
    return lagged.fillna(BASE_PRICE)

# calculating the rolling average prices and replace any missing values with the base price
def safe_rolling_mean(series, window):
    roll = series.rolling(window=window, min_periods=1).mean()
    return roll.fillna(BASE_PRICE)


# add lagged features for spot prices
if 'spot_price_elec_mw' in df.columns:
    df["spot_price_1h"] = safe_lag(df["spot_price_elec_mw"], 1)
    df["spot_price_24h"] = safe_lag(df["spot_price_elec_mw"], 24)
    df["spot_price_7d"] = safe_lag(df["spot_price_elec_mw"], 24*7)

    # adding rolling averages for spot prices
    df["prev_daily_avg_spot_price"] = safe_rolling_mean(df["spot_price_elec_mw"], 24)
    df["prev_week_avg_spot_price"] = safe_rolling_mean(df["spot_price_elec_mw"], 24*7)
    df["prev_month_avg_spot_price"] = safe_rolling_mean(df["spot_price_elec_mw"], 24*30)

#Creating season column 
def get_season(m):
    if m in [12,1,2]:
        return 'winter'
    if m in [3,4,5]:
        return 'spring'
    if m in [6,7,8]:
        return 'summer'
    return 'autumn'

df['season'] = df['month'].apply(get_season)

## One-hot encoding exogenous features

In [ ]:
#Creating one-hot encoded features for the models
categorical_cols = [c for c in ['school_zone', 'vacation_name', 'holiday_name', 'season'] if c in df.columns]
if categorical_cols:
    df[categorical_cols] = df[categorical_cols].astype('category')
    df = pd.get_dummies(df, columns=categorical_cols, prefix=categorical_cols)


## Fixing name consistency

In [ ]:
#normalizing the names for the columns
def clean_col(col):
    col = col.lower()
    col = col.replace(" ", "_")
    col = re.sub(r"[^\w\d_]", "", col)
    return col

df.columns = [clean_col(c) for c in df.columns]



## Additional fixing of features type

In [ ]:
#Giving categorization for insee reigon and weather code
for c in ['insee_region', 'weather_code']:
    if c in df.columns:
        df[c] = df[c].astype('category')

In [ ]:
#Making sure the dataset is optimalized so it is less computational expensive for the models to run 

int_cols = df.select_dtypes(include=['int64']).columns
if len(int_cols) > 0:
    df[int_cols] = df[int_cols].apply(pd.to_numeric, downcast='integer')

float_cols = df.select_dtypes(include=['float64']).columns
if len(float_cols) > 0:
    df[float_cols] = df[float_cols].astype('float32')

bool_cols = df.select_dtypes(include=['bool']).columns
if len(bool_cols) > 0:
    df[bool_cols] = df[bool_cols].astype('uint8')


In [ ]:
#Refixing a issue of column disposable income that is not properly tranferred
if 'disposable_income' in df.columns:
    df['disposable_income'] = pd.to_numeric(df['disposable_income'], errors='coerce').astype('float32')

df.rename(columns={'gdp_per employment': 'gdp_per_employment'}, inplace=True)

## Removing correlated feature

In [ ]:
#Removing columns after checking the correlation being above 0.8
keep_cols = [
    "datetime_hour", "insee_region", "conso_elec_mw", "conso_gaz_mw",
    "temperature_2m", "relative_humidity_2m", "pressure_msl", "surface_pressure",
    "rain", "snowfall", "cloud_cover", "cloud_cover_low", "cloud_cover_mid", "cloud_cover_high",
    "shortwave_radiation", "diffuse_radiation", "sunshine_duration",
    "wind_speed_10m", "wind_direction_10m", "weather_code", "snow_depth",
    "soil_moisture_0_to_7cm", "year", "quarter", "hour", "day_of_week", "month", "is_weekend"
]
keep_cols = [c for c in keep_cols if c in df.columns]

print("Duplicates included:", df.shape)
df = df.drop_duplicates().reset_index(drop=True)

## Filteren and saving the datasets for experiment

In [ ]:
#Filteren the years and checking the additional data with removing the correlared features
df_2015_2019 = df[(df['datetime_hour'].dt.year >= 2015) & (df['datetime_hour'].dt.year <= 2019)].copy()

cols_all_minus_desc = [c for c in df_2015_2019.columns if c not in ['region_name', 'weather_desc']]
df_additional = df_2015_2019[cols_all_minus_desc].copy()

cols_to_remove = ["region_name", "weather_desc",
                  "precipitation", "dew_point_2m", "apparent_temperature", "pressure_msl",
                  "wind_speed_100m", "wind_direction_100m", "wind_gusts_10m", "et0_fao_evapotranspiration"]
cols_to_remove = [c for c in cols_to_remove if c in df_additional.columns]

df_additional.drop(columns=cols_to_remove, inplace=True)
df_additional['datetime_hour'] = pd.to_datetime(df_additional['datetime_hour'])
df_additional['week_of_year'] = df_additional['datetime_hour'].dt.isocalendar().week.astype('int8')

In [ ]:
#Saving the datasets to use for further use

#Basic dataset
df_original_data = df_2015_2019[keep_cols + ['week_of_year']].copy()
df_original_data.to_csv("Hourly_Elec_Original_Data.csv", index=False, date_format='%Y-%m-%d %H:%M:%S')
df_original_data.to_pickle("Hourly_Elec_Original_Data.pkl")


#Enriched dataset
df_additional.to_csv("Hourly_Elec_Additional_Data.csv", index=False, date_format='%Y-%m-%d %H:%M:%S')
df_additional.to_pickle("Hourly_Elec_Additional_Data.pkl")

#EDA daataset
eda_extra = [c for c in ["region_name", "weather_desc"] if c in df_2015_2019.columns]
eda_cols = keep_cols + eda_extra

df_eda = df_2015_2019[eda_cols + ["week_of_year"]].copy()
df_eda.to_csv("Hourly_Elec_Original_EDA.csv", index=False, date_format='%Y-%m-%d %H:%M:%S')
df_eda.to_pickle("Hourly_Elec_Original_EDA.pkl")

